# eSewa (ePay v2) Integration — Colab Demo

This notebook shows a **working UAT demo** of eSewa ePay v2:
- Launch a public callback URL via **ngrok**
- Generate the **HMAC-SHA256** signature (`signature`, `signed_field_names`)
- Post an auto-submitting form to **eSewa UAT** (hosted checkout)
- Receive success/failure at your callback, then **verify** via status API

**Note:** This demo uses eSewa's *UAT credentials* and endpoints.

**Docs:** eSewa developer portal (signature & params) and test credentials.


In [1]:
!pip -q install flask pyngrok requests
print('Installed Flask, pyngrok, requests')

Installed Flask, pyngrok, requests


## 1) Configure UAT constants
Replace `PRODUCT_CODE` and `SECRET_KEY` with your live values when moving to production.

In [2]:
import os, base64, hmac, hashlib, json, time

# --- UAT test values from eSewa docs ---
PRODUCT_CODE = os.environ.get('ESEWA_PRODUCT_CODE', 'EPAYTEST')
SECRET_KEY   = os.environ.get('ESEWA_SECRET', '8gBm/:&EnhH.1/q')

# ePay v2 UAT form endpoint
ESEWA_FORM_URL = os.environ.get('ESEWA_FORM_URL', 'https://rc-epay.esewa.com.np/api/epay/main/v2/form')

# ePay v2 status check endpoint (UAT)
STATUS_API    = os.environ.get('ESEWA_STATUS_URL', 'https://uat.esewa.com.np/api/epay/transaction/status/')

print('PRODUCT_CODE=', PRODUCT_CODE)
print('FORM_URL    =', ESEWA_FORM_URL)
print('STATUS_API  =', STATUS_API)

PRODUCT_CODE= EPAYTEST
FORM_URL    = https://rc-epay.esewa.com.np/api/epay/main/v2/form
STATUS_API  = https://uat.esewa.com.np/api/epay/transaction/status/


## 2) Signature helper (HMAC-SHA256 → Base64)
Per eSewa v2: `signed_field_names` must **include and be ordered as** `total_amount,transaction_uuid,product_code`. We then compute the HMAC using the concatenated string of `key=value` pairs joined by commas, and Base64-encode the digest.

In [3]:
def esewa_signature(total_amount: str, transaction_uuid: str, product_code: str, secret_key: str) -> str:
    # Build the canonical string exactly in this order
    data = f"total_amount={total_amount},transaction_uuid={transaction_uuid},product_code={product_code}"
    digest = hmac.new(secret_key.encode(), data.encode(), hashlib.sha256).digest()
    return base64.b64encode(digest).decode()

# quick test
print(esewa_signature('100', '11-201-13', PRODUCT_CODE, SECRET_KEY))

5DZywcrTKD0gia/rsSMcrRHmJl+4Tbol6S+lWgdJ94E=


## 3) Start Flask with ngrok (public callbacks)
We expose `/success` and `/failure` to receive eSewa redirects. The success handler:
- Parses `data` (Base64 JSON from eSewa)
- Recomputes signature locally for integrity
- Calls **status API** to confirm transaction status

Then we create a `/pay` route that renders an auto-submitting form to eSewa using test values.

In [4]:
import threading
from flask import Flask, request, Response
from pyngrok import ngrok, conf
import requests
import uuid # Import the uuid library
from google.colab import userdata # Import userdata

app = Flask(__name__)

# Configure ngrok with the authtoken from Colab secrets
# Make sure you have added NGROK_AUTH_TOKEN as a secret in the left sidebar
try:
    ngrok_auth_token = userdata.get('NGROK_AUTH_TOKEN')
    if ngrok_auth_token:
        conf.get_default().auth_token = ngrok_auth_token
    else:
        print("NGROK_AUTH_TOKEN not found in Colab secrets. ngrok might not work correctly.")
except Exception as e:
    print(f"Error retrieving NGROK_AUTH_TOKEN from Colab secrets: {e}")


public_url = ngrok.connect(5000, bind_tls=True).public_url

@app.get('/')
def index():
    return f"<h3>eSewa v2 Demo</h3><p>Public URL: {public_url}</p><p>Go to <a href='{public_url}/pay' target='_blank'>/pay</a> to initiate a test payment.</p>"

@app.get('/pay')
def pay():
    # Inputs
    total_amount = request.args.get('total_amount')  # allow override
    amount = request.args.get('amount')              # base amount
    tax_amount = request.args.get('tax_amount', '0')
    psc = request.args.get('product_service_charge', '0')
    pdc = request.args.get('product_delivery_charge', '0')
    txn_uuid = request.args.get('txn', str(uuid.uuid4()))

    # Default: if only amount given, compute total_amount
    if amount is None and total_amount is None:
        amount = '100'  # demo default
    if total_amount is None:
        total_amount = str(round(float(amount) + float(tax_amount) + float(psc) + float(pdc), 2))

    signed_field_names = 'total_amount,transaction_uuid,product_code'
    sig = esewa_signature(total_amount, txn_uuid, PRODUCT_CODE, SECRET_KEY)

    form = f"""
    <h4>Auto-submitting to eSewa UAT…</h4>
    <form id='f' action='{ESEWA_FORM_URL}' method='POST'>
      <!-- v2 commercial fields -->
      <input type='hidden' name='amount' value='{amount}' />
      <input type='hidden' name='tax_amount' value='{tax_amount}' />
      <input type='hidden' name='product_service_charge' value='{psc}' />
      <input type='hidden' name='product_delivery_charge' value='{pdc}' />
      <input type='hidden' name='total_amount' value='{total_amount}' />

      <!-- v2 required identity/signature fields -->
      <input type='hidden' name='transaction_uuid' value='{txn_uuid}' />
      <input type='hidden' name='product_code' value='{PRODUCT_CODE}' />
      <input type='hidden' name='success_url' value='{public_url}/success' />
      <input type='hidden' name='failure_url' value='{public_url}/failure' />
      <input type='hidden' name='signed_field_names' value='{signed_field_names}' />
      <input type='hidden' name='signature' value='{sig}' />

      <noscript><button type='submit'>Pay with eSewa (UAT)</button></noscript>
    </form>
    <script>document.getElementById('f').submit();</script>
    <p><b>Txn UUID:</b> {txn_uuid}</p>
    <p><b>Amount breakdown:</b> amount={amount}, tax={tax_amount}, psc={psc}, pdc={pdc}, total={total_amount}</p>
    """
    return form


def decode_esewa_data(b64: str):
    try:
        raw = base64.b64decode(b64 + "==")  # tolerate missing padding
        obj = json.loads(raw.decode())
        return obj
    except Exception as e:
        return {"error": str(e)}

@app.get('/success')
def success():
    # eSewa v2 sends a 'data' param (Base64 JSON) containing fields such as
    # total_amount, transaction_uuid, product_code, reference_id, status, signed_field_names, signature
    b64 = request.args.get('data') or request.values.get('data')
    if not b64:
        return Response("Missing data", status=400)
    payload = decode_esewa_data(b64)
    if 'error' in payload:
        return Response(f"Decode error: {payload['error']}", status=400)

    # Integrity check: recompute signature
    try:
        ta = str(payload.get('total_amount'))
        tu = payload.get('transaction_uuid')
        pc = payload.get('product_code')
        srv_sig = payload.get('signature')
        calc_sig = esewa_signature(ta, tu, pc, SECRET_KEY)
        ok_sig = (srv_sig == calc_sig)
    except Exception as e:
        ok_sig = False

    # Status verification call
    params = {
        'product_code': pc,
        'total_amount': ta,
        'transaction_uuid': tu
    }
    try:
        r = requests.get(STATUS_API, params=params, timeout=15)
        status_json = r.json()
    except Exception as e:
        status_json = {"error": str(e)}

    return {
        "received": payload,
        "signature_match": ok_sig,
        "status_api_query": params,
        "status_api_response": status_json
    }

@app.get('/failure')
def failure():
    return {"message": "Payment failed", "query": dict(request.args)}

def run():
    app.run(host='0.0.0.0', port=5000)

thread = threading.Thread(target=run, daemon=True)
thread.start()
time.sleep(2)
public_url

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


'https://0333bdaa1bad.ngrok-free.app'

### Next steps
1. Click the link printed above (public URL), then open **`/pay`** to initiate a UAT payment.
2. Log in with **test eSewa accounts** (see the credentials in the eSewa docs).
3. After confirming payment, you’ll be redirected to **`/success`** here and the notebook will display:
   - Decoded payload (`data`)
   - Local signature match (`True/False`)
   - Response from **status API** (final confirmation)

**Tip:** You can pass `amount` and `txn` query params to `/pay`, e.g. `/pay?amount=150&txn=demo-123`.